In [26]:
import pandas as pd
import numpy as np


# 데이터 불러오기
train_data = pd.read_csv('wafer_TRAIN', header=None, sep=',')
test_data = pd.read_csv('wafer_TEST', header=None, sep=',')

# 긍정적 클래스(소수 클래스)의 샘플을 무작위로 선택하여 불균형 비율 조정
# 긍정적 클래스는 '-1', 부정적 클래스는 '1'로 가정 (논문의 설정에 따라서 다를 수 있음)
positive_class_label = -1
negative_class_label = 1

# 긍정적 클래스의 인덱스 추출
positive_indices = train_data[train_data.iloc[:, 0] == positive_class_label].index

# 긍정적 클래스 샘플 중 무작위로 선택 (예를 들어 52%만 유지)
selected_positive_indices = np.random.choice(positive_indices, size=int(len(positive_indices) * 0.52), replace=False)

# 불균형 데이터셋 생성
balanced_train_data = train_data.loc[selected_positive_indices].append(train_data[train_data.iloc[:, 0] == negative_class_label])

# 새로운 훈련 및 테스트 세트 분할
X_train, y_train = balanced_train_data.iloc[:, 1:], balanced_train_data.iloc[:, 0]
X_test, y_test = test_data.iloc[:, 1:], test_data.iloc[:, 0]

# 불균형 비율 확인
imbalance_ratio = len(y_train[y_train == negative_class_label]) / len(y_train[y_train == positive_class_label])
imbalance_ratio, X_train.shape, y_train.shape, X_test.shape, y_test.shape


<ipython-input-26-39acaac820f5>:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  balanced_train_data = train_data.loc[selected_positive_indices].append(train_data[train_data.iloc[:, 0] == negative_class_label])


(18.06, (953, 152), (953,), (6164, 152), (6164,))

In [27]:
# positive 개수 , Imbalance ratio 맞춤
(y_train==-1).sum()

50

In [31]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, f1_score, precision_recall_curve, auc
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline

# 데이터셋 준비
# X_train, y_train, X_test, y_test = ...

# 파이프라인 생성: 스케일링, SMOTE, 분류기 순으로 처리
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # 데이터 스케일링
    ('smote', SMOTE(random_state=42)),  # SMOTE 오버샘플링
    ('classifier', RandomForestClassifier(random_state=42))  # 분류기
])

# 모델 훈련
pipeline.fit(X_train, y_train)

# 테스트 데이터에 대한 예측 수행
y_pred = pipeline.predict(X_test)
y_pred_proba = pipeline.predict_proba(X_test)[:, 1]  # 양성 클래스에 대한 확률

# 성능 지표 계산
auc_score = roc_auc_score(y_test, y_pred_proba)
f1 = f1_score(y_test, y_pred)
precision, recall, _ = precision_recall_curve(y_test, y_pred_proba)
auprc = auc(recall, precision)

# 결과 출력
print(f"AUC: {auc_score}")
print(f"F1 Score: {f1}")
print(f"AUPRC: {auprc}")


AUC: 0.9990649017524716
F1 Score: 0.9935600907029478
AUPRC: 0.9998898793253912
